# Loop 14 Analysis: Rotation Optimization

## Key Findings from 14 Experiments:
1. All local optimization approaches (SA, GA, fractional translation) found ZERO improvements
2. The baseline is at a TRUE local optimum
3. "Rebuild from corners" found only numerical noise improvements
4. External data sources have WORSE scores than our baseline

## New Approach: Rotation Optimization
The bbox3 runner kernel uses scipy.optimize.minimize_scalar to find optimal rotation angle for entire group.
This can tighten bounding boxes without changing tree positions.

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize_scalar
import time

# Load baseline
baseline_path = '/home/nonroot/snapshots/santa-2025/21337353543/submission/submission.csv'
df = pd.read_csv(baseline_path)

# Tree polygon vertices
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_tree_vertices(x, y, angle_deg):
    angle_rad = np.radians(angle_deg)
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)
    rx = TX * cos_a - TY * sin_a + x
    ry = TX * sin_a + TY * cos_a + y
    return rx, ry

def compute_bbox_side(trees_x, trees_y, trees_angle):
    all_x = []
    all_y = []
    for x, y, a in zip(trees_x, trees_y, trees_angle):
        vx, vy = get_tree_vertices(x, y, a)
        all_x.extend(vx)
        all_y.extend(vy)
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def rotate_all_trees(trees_x, trees_y, trees_angle, rotation_deg):
    """Rotate entire layout by rotation_deg degrees around centroid."""
    # Find centroid
    cx = np.mean(trees_x)
    cy = np.mean(trees_y)
    
    # Rotate positions around centroid
    angle_rad = np.radians(rotation_deg)
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)
    
    new_x = (trees_x - cx) * cos_a - (trees_y - cy) * sin_a + cx
    new_y = (trees_x - cx) * sin_a + (trees_y - cy) * cos_a + cy
    new_angle = trees_angle + rotation_deg
    
    return new_x, new_y, new_angle

print('Functions defined. Testing on N=10...')

In [ ]:
# Load solutions for a specific N
def load_n(df, n):
    n_df = df[df['id'].str.startswith(f'{n:03d}_')]
    trees_x = n_df['x'].str.replace('s', '').astype(float).values
    trees_y = n_df['y'].str.replace('s', '').astype(float).values
    trees_angle = n_df['deg'].str.replace('s', '').astype(float).values
    return trees_x, trees_y, trees_angle

# Test rotation optimization on N=10
n = 10
trees_x, trees_y, trees_angle = load_n(df, n)
baseline_side = compute_bbox_side(trees_x, trees_y, trees_angle)
baseline_score = (baseline_side ** 2) / n

print(f'N={n}: Baseline side={baseline_side:.8f}, score={baseline_score:.8f}')

# Find optimal rotation
def score_at_rotation(rotation_deg):
    new_x, new_y, new_a = rotate_all_trees(trees_x, trees_y, trees_angle, rotation_deg)
    side = compute_bbox_side(new_x, new_y, new_a)
    return (side ** 2) / n

result = minimize_scalar(score_at_rotation, bounds=(-45, 45), method='bounded')
print(f'Optimal rotation: {result.x:.4f} degrees')
print(f'Optimized score: {result.fun:.8f}')
print(f'Improvement: {baseline_score - result.fun:.10f}')

In [ ]:
# Test rotation optimization on multiple N values
test_ns = [5, 10, 20, 30, 50, 100, 150, 200]
results = []

for n in test_ns:
    trees_x, trees_y, trees_angle = load_n(df, n)
    baseline_side = compute_bbox_side(trees_x, trees_y, trees_angle)
    baseline_score = (baseline_side ** 2) / n
    
    def score_at_rotation(rotation_deg):
        new_x, new_y, new_a = rotate_all_trees(trees_x, trees_y, trees_angle, rotation_deg)
        side = compute_bbox_side(new_x, new_y, new_a)
        return (side ** 2) / n
    
    result = minimize_scalar(score_at_rotation, bounds=(-45, 45), method='bounded')
    improvement = baseline_score - result.fun
    
    results.append({
        'n': n,
        'baseline_score': baseline_score,
        'optimized_score': result.fun,
        'optimal_rotation': result.x,
        'improvement': improvement
    })
    
    if improvement > 1e-8:
        print(f'N={n}: IMPROVED by {improvement:.10f} (rotation={result.x:.4f}°)')
    else:
        print(f'N={n}: No improvement (rotation={result.x:.4f}°)')

print(f'\nTotal improvement: {sum(r["improvement"] for r in results):.10f}')

In [ ]:
# Check ALL N values for rotation optimization
print('Testing rotation optimization on ALL N values (1-200)...')
start_time = time.time()

all_results = []
improvements = []

for n in range(1, 201):
    trees_x, trees_y, trees_angle = load_n(df, n)
    baseline_side = compute_bbox_side(trees_x, trees_y, trees_angle)
    baseline_score = (baseline_side ** 2) / n
    
    def score_at_rotation(rotation_deg):
        new_x, new_y, new_a = rotate_all_trees(trees_x, trees_y, trees_angle, rotation_deg)
        side = compute_bbox_side(new_x, new_y, new_a)
        return (side ** 2) / n
    
    result = minimize_scalar(score_at_rotation, bounds=(-45, 45), method='bounded')
    improvement = baseline_score - result.fun
    
    all_results.append({
        'n': n,
        'baseline_score': baseline_score,
        'optimized_score': result.fun,
        'optimal_rotation': result.x,
        'improvement': improvement
    })
    
    if improvement > 1e-8:
        improvements.append((n, improvement, result.x))

elapsed = time.time() - start_time
print(f'\nCompleted in {elapsed:.2f}s')
print(f'N values with improvements: {len(improvements)}')
print(f'Total improvement: {sum(r["improvement"] for r in all_results):.10f}')

if improvements:
    print('\nTop 10 improvements:')
    improvements.sort(key=lambda x: -x[1])
    for n, imp, rot in improvements[:10]:
        print(f'  N={n}: +{imp:.10f} (rotation={rot:.4f}°)')